In [4]:
import os 
import datetime
import json
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import tensorflow as tf

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())
basepath = Path(os.getcwd())

%load_ext autoreload
%autoreload 2
basepath

PosixPath('/srv/idp-radio-1')

In [5]:
# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

#config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 1.2


In [6]:
tf.compat.v1.Session(config=config)
tf.__version__

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5



'2.4.0-dev20200815'

In [34]:
import numpy as np

import traceback
from sklearn.metrics import classification_report
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam, SGD
from keras.utils.generic_utils import get_custom_objects

from tensorflow.keras.applications import InceptionV3, Xception, DenseNet121, InceptionResNetV2, ResNet152V2, NASNetLarge
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.simple.load_model import get_all_experiment_logs, get_experiment_from_logs, benchmark_from_logs, get_preprocessing_for_architecture, get_model_build_function, rebuild_experiment, difference_test_results, reevaluate_on_datagen
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from src.architectures.benchmarks.benchmark_definitions import generate_benchmarks,simple_architecture_experiment, Chexpert_Benchmark, CHEXPERT_COLUMNS, METRICS, SINGLE_CLASS_METRICS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight
from src.utils.save_model import get_experiment, load_model
from src.metrics.metrics import SingleClassMetric, NaNWrapper, F2Score, FBetaScore
import tensorflow

In [8]:
experiments = get_all_experiment_logs()
experiments = [exp for exp in experiments if "Failed" not in exp["name"]]
#experiments = [exp for exp in experiments if "N12" in exp["name"]]
experiments = [exp for exp in experiments if "num_samples_test" in exp["benchmark"].keys() ]
experiments = [exp for exp in experiments if exp["benchmark"]["num_samples_test"] == 234 ]
len(experiments)

42

In [45]:
exp_dict = experiments[-1].copy()
exp_dict["name"], exp_dict["test"]["auc"]

('DenseNet121_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp51_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled',
 0.7465326189994812)

In [48]:
exp = rebuild_experiment(exp_dict)

Using weights file DenseNet121_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp51_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled_20200906-065845.h5 to load model...


In [49]:
def reevaluate_aug(exp, average_over=10, new_metrics=True):
    
    testgen = exp.benchmark.testgen
    testgen.augmentation = exp.benchmark.traingen.augmentation
    
    preds = np.zeros((10, len(testgen), 12))
    for i in range(10):
        preds[i] = exp.model.predict(
            testgen, steps=len(testgen), verbose=1)
    
    preds = np.average(preds, axis=0)
    
    exp.groundtruth_label = testgen.get_labels_nonan()
    
    metric_results = {}
    if new_metrics:
        metrics = []
        for m in exp.benchmark.metrics:
            if isinstance(m, SingleClassMetric):
                class_name = "_".join(m.name.split("_")[1:])
                metrics.append(SingleClassMetric(m.base_metric, m.class_id, class_name= class_name))
            else:
                extra_args = {}
                if hasattr(m, "multi_label"):
                    extra_args["multi_label"] = m.multi_label
                               
                metrics.append(m.__class__(name= m.name, **extra_args))
    else:
        metrics = exp.benchmark.metrics
    
    for metric in metrics:
        metric.reset_states()
        metric.update_state(tensorflow.constant(exp.groundtruth_label), tensorflow.constant(preds))
        metric_results[metric.name] = metric.result().numpy()
        
    return metric_results

In [50]:
res = reevaluate_aug(exp)

234/234 [==============================] - 4s 18ms/step


In [51]:
difference_test_results(res, exp_dict["test"])

{'auc': 0.00934445858001709,
 'precision': 0.02370131015777588,
 'recall': 0.003663003444671631,
 'f2_score': 0.004427388310432434,
 'binary_accuracy': 0.0010682344436645508,
 'accuracy_enlarged_cardiomediastinum': 0.0,
 'accuracy_cardiomegaly': 0.017094016075134277,
 'accuracy_lung_opacity': 0.0,
 'accuracy_lung_lesion': 0.004273474216461182,
 'accuracy_edema': 0.0,
 'accuracy_consolidation': 0.0,
 'accuracy_pneumonia': 0.0,
 'accuracy_atelectasis': 0.0,
 'accuracy_pneumothorax': 0.0,
 'accuracy_pleural_effusion': 0.008547008037567139,
 'accuracy_pleural_other': 0.008547008037567139,
 'accuracy_fracture': 0.0,
 'auc_enlarged_cardiomediastinum': 0.02763301134109497,
 'auc_cardiomegaly': 0.010364949703216553,
 'auc_lung_opacity': 0.0033068060874938965,
 'auc_lung_lesion': 0.18454933166503906,
 'auc_edema': 0.00540846586227417,
 'auc_consolidation': 0.0011307597160339355,
 'auc_pneumonia': 0.020741164684295654,
 'auc_atelectasis': 0.00831979513168335,
 'auc_pneumothorax': 0.0464601516723

In [52]:
res["auc"], exp_dict["test"]["auc"]

(0.73718816, 0.7465326189994812)